In [6]:
# union all 1x2_odds files together and join to master data


In [7]:
# imports
import pandas as pd

In [8]:
# get 1x2 odds data for last few years
games_0 = pd.read_excel('data/2023_1x2_data.xlsx', header=0)
games_1 = pd.read_excel('data/2024_1x2_data.xlsx', header=0)
games_2 = pd.read_excel('data/2025_1x2_data.xlsx', header=0)

# union together
all_1x2_odds = pd.concat([games_0, games_1, games_2], ignore_index=True)

# reset index
all_1x2_odds.reset_index(drop=True, inplace=True)

# only keep key cols
key_cols = ['Game_Link'] + [x for x in all_1x2_odds.columns if x.startswith('Odds')]
all_1x2_odds = all_1x2_odds[key_cols]

# inspect
all_1x2_odds.info()
all_1x2_odds.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Game_Link  3349 non-null   object 
 1   Odds_1     3224 non-null   float64
 2   Odds_X     3224 non-null   float64
 3   Odds_2     3224 non-null   float64
dtypes: float64(3), object(1)
memory usage: 104.8+ KB


,Game_Link,Odds_1,Odds_X,Odds_2
0,https://www.flashscore.com/match/hockey/edmont...,2.45,4.0,2.45
1,https://www.flashscore.com/match/hockey/edmont...,2.35,4.0,2.55
2,https://www.flashscore.com/match/hockey/edmont...,2.00,4.0,3.10
3,https://www.flashscore.com/match/hockey/edmont...,2.30,4.0,2.55
4,https://www.flashscore.com/match/hockey/edmont...,2.15,4.0,2.80


In [9]:
# read in master games data
all_games = pd.read_excel(r'data/master_games_data.xlsx', header=0)

# inspect
all_games.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3381 entries, 0 to 3380
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Game_Link                 3381 non-null   object        
 1   Extra_Time                3381 non-null   object        
 2   Home_Team                 3381 non-null   object        
 3   Away_Team                 3381 non-null   object        
 4   Home_Score                3381 non-null   int64         
 5   Away_Score                3381 non-null   int64         
 6   P1_Home_Score             3381 non-null   int64         
 7   P1_Away_Score             3381 non-null   int64         
 8   P2_Home_Score             3381 non-null   int64         
 9   P2_Away_Score             3381 non-null   int64         
 10  P3_Home_Score             3381 non-null   int64         
 11  P3_Away_Score             3381 non-null   int64         
 12  P4_Home_Score       

In [10]:
# join the 2 dfs together
master_data = pd.merge(all_games, all_1x2_odds, how='left', on='Game_Link')

# inspect
master_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3381 entries, 0 to 3380
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Game_Link                 3381 non-null   object        
 1   Extra_Time                3381 non-null   object        
 2   Home_Team                 3381 non-null   object        
 3   Away_Team                 3381 non-null   object        
 4   Home_Score                3381 non-null   int64         
 5   Away_Score                3381 non-null   int64         
 6   P1_Home_Score             3381 non-null   int64         
 7   P1_Away_Score             3381 non-null   int64         
 8   P2_Home_Score             3381 non-null   int64         
 9   P2_Away_Score             3381 non-null   int64         
 10  P3_Home_Score             3381 non-null   int64         
 11  P3_Away_Score             3381 non-null   int64         
 12  P4_Home_Score       

In [13]:
# function that takes the df, sorts it by the target grain, then performs the groupby
def grain_sort_cumsum_props(home_vs_away: str, df: pd.DataFrame) -> pd.DataFrame:
    # add home or away team to sort grain
    home_plus_year = [home_vs_away, 'Season']
    
    # combine to create final sort grain
    sort_grain = home_plus_year + ['Date', 'Game_ID'] 

    # sort
    df = df.sort_values(by=sort_grain)

    # hard code cumsum cols
    cum_cols = ["Reg_Home_Win", "Reg_Away_Win", "Reg_Tie", 'P1_Home_Score', 'P1_Away_Score', 
                'P2_Home_Score', 'P2_Away_Score', 'P3_Home_Score', 'P3_Away_Score',
                'regular_time_score_home', 'regular_time_score_away']
    
    # add count of games for home or away
    df[f'cum_{home_vs_away}_Games'] = df.groupby(home_plus_year).cumcount() + 1

    # compute cumsum for each group
    df[[f"cum_{col}" for col in cum_cols]] = df.groupby(home_plus_year)[cum_cols].cumsum()

    # calc prop home reg win
    df['prop_Reg_Home_Win'] = df['cum_Reg_Home_Win'] / df[f'cum_{home_vs_away}_Games']
    df['prop_Reg_Away_Win'] = df['cum_Reg_Away_Win'] / df[f'cum_{home_vs_away}_Games']
    df['prop_Reg_Tie'] = df['cum_Reg_Tie'] / df[f'cum_{home_vs_away}_Games']

    # calc props for what % of goals scored in each period
    df['prop_P1_Home_Score'] = df['cum_P1_Home_Score'] / df['cum_regular_time_score_home']
    df['prop_P2_Home_Score'] = df['cum_P2_Home_Score'] / df['cum_regular_time_score_home']
    df['prop_P3_Home_Score'] = df['cum_P3_Home_Score'] / df['cum_regular_time_score_home']

    df['prop_P1_Away_Score'] = df['cum_P1_Away_Score'] / df['cum_regular_time_score_away']
    df['prop_P2_Away_Score'] = df['cum_P2_Away_Score'] / df['cum_regular_time_score_away']
    df['prop_P3_Away_Score'] = df['cum_P3_Away_Score'] / df['cum_regular_time_score_away']

    # calc goal diffs at the period level
    df['prop_P1_Home_Goal_Diff'] = df['cum_P1_Home_Score'] / df['cum_P1_Away_Score']
    df['prop_P2_Home_Goal_Diff'] = df['cum_P2_Home_Score'] / df['cum_P2_Away_Score']
    df['prop_P3_Home_Goal_Diff'] = df['cum_P3_Home_Score'] / df['cum_P3_Away_Score']

    df['prop_P1_Away_Goal_Diff'] = df['cum_P1_Away_Score'] / df['cum_P1_Home_Score']
    df['prop_P2_Away_Goal_Diff'] = df['cum_P2_Away_Score'] / df['cum_P2_Home_Score']
    df['prop_P3_Away_Goal_Diff'] = df['cum_P3_Away_Score'] / df['cum_P3_Home_Score']

    # calc props for goals scored / allowed
    df['prop_reg_home_goal_diff'] = df['cum_regular_time_score_home'] / (df['cum_regular_time_score_home'] + df['cum_regular_time_score_away'])
    df['prop_reg_away_goal_diff'] = 1 - df['prop_reg_home_goal_diff']
    
    return df

# test func
test_all_home_games = grain_sort_cumsum_props('Home_Team', master_data)

# isolate cumcols
sel_cum_cols = [x for x in test_all_home_games.columns if x.startswith('cum_') or x.startswith('prop_')]

# add date, gid, and home team to list
peek_cols = ['Date', 'Season', 'Game_ID', 'Home_Team'] + sel_cum_cols

# inspect cum cols
test_all_home_games[peek_cols].head(5)

,Date,Season,Game_ID,Home_Team,cum_Home_Team_Games,cum_Reg_Home_Win,cum_Reg_Away_Win,cum_Reg_Tie,cum_P1_Home_Score,cum_P1_Away_Score,...,prop_P2_Away_Score,prop_P3_Away_Score,prop_P1_Home_Goal_Diff,prop_P2_Home_Goal_Diff,prop_P3_Home_Goal_Diff,prop_P1_Away_Goal_Diff,prop_P2_Away_Goal_Diff,prop_P3_Away_Goal_Diff,prop_reg_home_goal_diff,prop_reg_away_goal_diff
1496,2023-09-24,2023,2023-09-24-20:00|Anaheim Ducks vs Los Angeles ...,Anaheim Ducks,1,0,0,1,1,0,...,0.500000,0.500000,inf,0.0,1.000000,0.000000,inf,1.000000,0.500000,0.500000
1471,2023-09-27,2023,2023-09-27-22:00|Anaheim Ducks vs San Jose Sharks,Anaheim Ducks,2,1,0,1,3,1,...,0.250000,0.500000,3.00,1.0,1.000000,0.333333,1.000000,1.000000,0.600000,0.400000
1457,2023-09-29,2023,2023-09-29-22:00|Anaheim Ducks vs Los Angeles ...,Anaheim Ducks,3,1,1,1,4,4,...,0.250000,0.250000,1.00,1.0,1.500000,1.000000,1.000000,0.666667,0.529412,0.470588
1413,2023-10-05,2023,2023-10-05-22:00|Anaheim Ducks vs Arizona Coyotes,Anaheim Ducks,4,1,2,1,4,4,...,0.333333,0.333333,1.00,0.5,1.250000,1.000000,2.000000,0.800000,0.478261,0.521739
1366,2023-10-15,2023,2023-10-15-20:30|Anaheim Ducks vs Carolina Hur...,Anaheim Ducks,5,2,2,1,7,4,...,0.333333,0.400000,1.75,0.6,1.166667,0.571429,1.666667,0.857143,0.531250,0.468750


In [14]:
# TODO - perform EDA on records that resulted in a regular time tie and see if there's any patterns or trends in this subset that could be indicative of predictive features
master_data[master_data['Reg_Tie'] == True].describe()

,Home_Score,Away_Score,P1_Home_Score,P1_Away_Score,P2_Home_Score,P2_Away_Score,P3_Home_Score,P3_Away_Score,P4_Home_Score,P4_Away_Score,...,Season,Total_Score,regular_time_score_home,regular_time_score_away,regular_time_score_total,Week_of_Year,Game_Start_Hour,Odds_1,Odds_X,Odds_2
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,...,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,678.000000,678.000000,678.000000
mean,3.137255,3.075630,0.809524,0.810924,0.924370,0.911765,0.872549,0.883754,0.385154,0.334734,...,2023.694678,6.212885,2.606443,2.606443,5.212885,28.043417,19.222689,2.324646,4.291512,2.900258
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2023.000000,1.000000,0.000000,0.000000,0.000000,1.000000,8.000000,1.290000,3.800000,1.380000
25%,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2023.000000,5.000000,2.000000,2.000000,4.000000,10.000000,19.000000,1.842500,4.100000,2.250000
50%,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,2024.000000,7.000000,3.000000,3.000000,6.000000,39.000000,19.000000,2.182500,4.225000,2.700000
75%,4.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2024.000000,7.000000,3.000000,3.000000,6.000000,45.000000,21.000000,2.625000,4.425000,3.300000
max,8.000000,7.000000,5.000000,4.000000,4.000000,4.000000,5.000000,5.000000,1.000000,1.000000,...,2025.000000,15.000000,7.000000,7.000000,14.000000,52.000000,22.000000,6.125000,6.000000,8.500000
std,1.246903,1.277662,0.823988,0.828553,0.881408,0.873035,0.866553,0.845209,0.486973,0.472228,...,0.689980,2.318788,1.159394,1.159394,2.318788,18.216543,2.246890,0.666943,0.293217,0.923385


In [ ]:
# TODO - think about what features could be indicative/relevent for identifying matchups that would require some free hockey to settle things

# the team's conf, div, and/or overall rank; in order to calc these ranks need to calc the number of wins at that grain; another way to think about it is total goals scored vs allowed

In [ ]:
# TODO - create cumulative teams table where each row is at the team + date level and I get cumulative:
# need to agg @ weekly level and calc weekly level ranks for simplicity vs daily ranks
# conf wins/losses, division wins/losses, conf rank, div rank
